In [1]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import seaborn as sns # SEABORN
import scipy
import sklearn
from sklearn.decomposition import PCA
import cv2
import os 
from skimage import filters
from scipy import ndimage
import math 


In [2]:
import glob
from pathlib import Path

In [3]:
%run auxfunctions.ipynb

In [33]:
path_to_test_emotion = '../ck+copynewT/Emotion_labels/Emotion/' #donde estan las secuencias de entrenamiento

In [24]:
test_series = glob.glob(mypath + '*/*/', recursive = True)
test_series = np.sort(test_series)

In [40]:
#fear == 4, sadness == 6 
series_fear = []
series_sadness = []
for i in np.sort(test_series):   
    actual_subject = str(i).split('/')[4]
    actual_subfolder = str(i).split('/')[5]
    actual_serie = actual_subject + '/'+ actual_subfolder
    
    emo = os.listdir(path_to_test_emotion + actual_serie)[0]
    emo_file = path_to_test_emotion + actual_serie + '/' + emo
    with open(emo_file) as f:
        emotion = int(float(f.readline()))
        if emotion == int(4):
            series_fear.append(actual_serie)
        elif emotion == int(6):
            series_sadness.append(actual_serie)
            
        

In [52]:
series_fear

['S011/003',
 'S032/004',
 'S050/001',
 'S054/002',
 'S055/006',
 'S059/002',
 'S065/002',
 'S068/004',
 'S074/001',
 'S091/001',
 'S102/003',
 'S117/003',
 'S119/003',
 'S124/003',
 'S125/006',
 'S128/003',
 'S129/004',
 'S506/004',
 'S999/003']

In [50]:
original_ckpathlandmarks = '../../ck+/Landmarks/'

np.sort(os.listdir(original_ckpathlandmarks))

array(['S005', 'S010', 'S011', 'S014', 'S022', 'S026', 'S028', 'S029',
       'S032', 'S034', 'S035', 'S037', 'S042', 'S044', 'S045', 'S046',
       'S050', 'S051', 'S052', 'S053', 'S054', 'S055', 'S056', 'S057',
       'S058', 'S059', 'S060', 'S061', 'S062', 'S063', 'S064', 'S065',
       'S066', 'S067', 'S068', 'S069', 'S070', 'S071', 'S072', 'S073',
       'S074', 'S075', 'S076', 'S077', 'S078', 'S079', 'S080', 'S081',
       'S082', 'S083', 'S084', 'S085', 'S086', 'S087', 'S088', 'S089',
       'S090', 'S091', 'S092', 'S093', 'S094', 'S095', 'S096', 'S097',
       'S098', 'S099', 'S100', 'S101', 'S102', 'S103', 'S104', 'S105',
       'S106', 'S107', 'S108', 'S109', 'S110', 'S111', 'S112', 'S113',
       'S114', 'S115', 'S116', 'S117', 'S118', 'S119', 'S120', 'S121',
       'S122', 'S124', 'S125', 'S126', 'S127', 'S128', 'S129', 'S130',
       'S131', 'S132', 'S133', 'S134', 'S135', 'S136', 'S137', 'S138',
       'S139', 'S147', 'S148', 'S149', 'S151', 'S154', 'S155', 'S156',
      

In [99]:
pca_shape =0
std_scaler =0

In [103]:
#serie_landmarks, shape_test_vector, serie_shape_data  = get_shape_features(series_fear, pca_shape, std_scaler)

#lo dejo asi, puede ser que requiera fit_transform. No sabemos.
#Ver ambas maneras, por ahora transform (Fit csv con fit)
#x, y, z = shape_test_vector.shape

#serie_landmarks_sadness, shape_test_vector_sadness, serie_shape_data_sadness  = get_shape_features(series_sadness, pca_shape, std_scaler)

#a, b, c = serie_shape_data.shape
#shape_test_vector =shape_test_vector.reshape(x,z)
#serie_shape_data = serie_shape_data.reshape(a,c)

a, b, c = serie_shape_data_sadness.shape
#shape_test_vector =shape_test_vector.reshape(x,z)
serie_shape_data_sadness = serie_shape_data_sadness.reshape(a,c)
serie_shape_data_sadness.shape

(21, 136)

In [107]:
my_df = pd.DataFrame(serie_shape_data_sadness)
my_df.to_csv('my_training_data_aligned_diff_newT_sadness.csv',header = False, index= False)

In [85]:
my_df = pd.DataFrame(serie_shape_data)
my_df.to_csv('my_training_data_aligned_diff_newT_fear.csv',header = False, index= False)

In [112]:
fear_data_forPCA = pd.read_csv('my_training_data_aligned_diff_newT_fear.csv',header = None)
sadness_data_forPCA = pd.read_csv('my_training_data_aligned_diff_newT_sadness.csv',header = None)

In [113]:
my_data_forPCA = pd.read_csv('my_training_data_aligned_diff_newT.csv',header = None)

In [114]:
old_and_sadfear_PCA = pd.concat([my_data_forPCA, fear_data_forPCA, sadness_data_forPCA ])

In [115]:
old_and_sadfear_PCA

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.321848,3.389416,3.614584,3.923668,4.586144,1.836830,1.209116,0.736995,0.059171,-0.597565,...,6.312549,4.202433,0.698461,-2.723177,-4.700435,-2.989341,1.120734,5.387435,7.346126,5.497563
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-3.865982,-4.154327,-4.198715,-2.676183,-2.193575,-1.075355,0.691931,1.080560,2.856966,3.918157,...,-15.640824,-8.321081,-5.836411,-5.513324,-3.785196,-4.675919,-4.567765,-5.097905,-4.901998,-5.592174
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,1.458526,1.438414,1.292610,0.785171,0.678849,0.627384,0.205456,-0.301711,-0.866286,-0.955327,...,0.108439,2.931707,4.997043,1.893707,2.080429,2.744648,8.619545,2.860622,2.164940,1.944843
17,-0.104826,-0.562747,-0.686865,-0.680543,0.646702,1.348654,-0.571505,-0.734606,-0.255969,0.191786,...,0.166520,-3.504449,-5.111879,-4.891640,1.355560,-5.638686,-5.644174,-5.812976,-5.715438,-5.178277
18,0.076568,-0.121391,0.034190,0.899219,1.997878,2.755954,0.482431,2.264971,3.470655,2.991495,...,-0.469402,1.088381,1.876698,1.847140,1.842480,1.967724,2.082946,2.128166,1.858659,2.027230
19,-1.035483,-1.319794,-0.520224,-0.638861,-0.768327,-1.485115,-1.991036,-2.130196,-1.617732,-0.482934,...,-11.757692,-6.985962,-4.470720,-4.595483,-6.648101,-4.667502,-4.617455,-4.725245,-6.656655,-4.192392


In [116]:
my_df.to_csv('my_training_data_aligned_diff_newT_fearSadness.csv',header = False, index= False)


In [121]:
pca_struct =0
std_scaler =0

In [130]:
struct_test_data_fear  = get_struct_features(series_fear, pca_struct, std_scaler,serie_landmarks, serie_shape_data)

In [132]:
struct_test_data_fear.shape

(19, 22)

In [133]:
my_df = pd.DataFrame(struct_test_data_fear)
my_df.to_csv('structure_feature_newT_vector_fear.csv',header = False, index= False)

In [139]:
struct_test_data_sadness = get_struct_features(series_sadness, pca_struct, std_scaler,serie_landmarks_sadness, serie_shape_data_sadness)

In [142]:
my_df = pd.DataFrame(struct_test_data_sadness)
my_df.to_csv('structure_feature_newT_vector_sadness.csv',header = False, index= False)

In [143]:
fear_struct_forPCA = pd.read_csv('structure_feature_newT_vector_fear.csv',header = None)
sadness_struct_forPCA = pd.read_csv('structure_feature_newT_vector_sadness.csv',header = None)

In [144]:
my_struct_forPCA = pd.read_csv('structure_feature_newT_vector.csv',header = None)

In [150]:
old_and_sadfear_PCA_struct = pd.concat([my_struct_forPCA, fear_struct_forPCA, sadness_struct_forPCA ])

In [151]:
old_and_sadfear_PCA_struct

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,178.322800,228.563070,223.199770,200.493930,117.806540,48.861560,41.303760,113.565385,56.161860,59.063270,...,114.590580,29.215750,23.671367,83.422690,55.054520,19885.578719,1.435555,1.304535,7.575994,0.000000
1,178.322800,228.563070,223.199770,200.493930,117.806540,48.861560,41.303760,113.565385,56.161860,59.063270,...,114.590580,29.215750,23.671367,83.422690,55.054520,19885.578719,1.435555,1.304535,7.575994,-0.053777
2,186.351832,245.849407,240.387553,220.659269,125.510782,49.671027,32.696519,127.791971,47.294834,80.218588,...,107.501807,25.145329,17.294111,52.715011,71.592272,19834.155421,1.429312,1.166385,7.971896,0.000000
3,186.351832,245.849407,240.387553,220.659269,125.510782,49.671027,32.696519,127.791971,47.294834,80.218588,...,107.501807,25.145329,17.294111,52.715011,71.592272,19834.155421,1.429312,1.166385,7.971896,0.056505
4,189.722463,249.013017,244.823872,223.735687,128.056980,50.491604,32.815432,124.547537,52.183083,71.828807,...,107.694540,25.195928,19.438491,54.397276,68.502287,18643.253452,1.424823,1.237680,8.351419,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,194.456477,275.915001,271.902666,240.508919,129.324025,55.278653,31.489335,110.033028,53.631778,57.525510,...,123.674258,41.018509,20.681080,53.995153,80.242647,14897.595425,1.455644,1.066826,5.648568,-0.059191
17,195.035231,259.547456,255.434053,227.563832,128.013536,72.363259,53.102772,111.542037,53.940240,61.048424,...,122.318460,34.531849,14.709116,63.291973,79.686798,18719.759710,1.483492,1.159776,8.614919,0.016853
18,188.553687,281.767023,277.329757,247.110867,134.525932,60.986409,55.352717,127.005979,58.823165,70.094316,...,113.103976,26.982421,16.405061,45.941228,78.247102,18298.252311,1.588546,0.992907,9.262380,-0.041790
19,170.732407,285.843562,280.859768,239.681172,132.244271,48.651397,42.980748,117.073367,47.146985,71.708133,...,127.940713,40.545784,12.679429,43.682013,93.143949,14783.402751,1.554418,1.035792,6.687458,-0.002796


In [152]:
my_df.to_csv('structure_feature_newT_vector_fearSadness.csv',header = False, index= False)


In [157]:
pca_cartoon = 0
std_scaler = 0

In [176]:
series_fear

['S011/003',
 'S032/004',
 'S050/001',
 'S054/002',
 'S055/006',
 'S059/002',
 'S065/002',
 'S068/004',
 'S074/001',
 'S091/001',
 'S102/003',
 'S117/003',
 'S119/003',
 'S124/003',
 'S125/006',
 'S128/003',
 'S129/004',
 'S506/004',
 'S999/003']

In [193]:
cartoon_test_vector_fear  = get_cartoon_features(series_fear, pca_cartoon, std_scaler)

(410, 410)
['ck+copynewTfile/cartoon_training_fear/S011/003/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_fear/S011/003/cartoonimg12.png']
(410, 410)
['ck+copynewTfile/cartoon_training_fear/S032/004/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_fear/S032/004/cartoonimg12.png']
(410, 410)
['ck+copynewTfile/cartoon_training_fear/S050/001/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_fear/S050/001/cartoonimg15.png']
(410, 410)
['ck+copynewTfile/cartoon_training_fear/S054/002/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_fear/S054/002/cartoonimg13.png']
(410, 410)
['ck+copynewTfile/cartoon_training_fear/S055/006/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_fear/S055/006/cartoonimg6.png']
(410, 410)
['ck+copynewTfile/cartoon_training_fear/S059/002/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_fear/S059/002/cartoonimg15.png']
(410, 410)
['ck+copynewTfile/cartoon_training_fear/S065/002/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_fear/S065/002/carto

In [196]:
a, b , c = cartoon_test_vector_fear.shape
cartoon_test_vector_fear = cartoon_test_vector_fear.reshape(a,c)

In [198]:
cartoon_test_vector_fear.shape

(19, 168100)

In [199]:
np.savetxt('im_newT_cartoonarray_fear.csv', cartoon_test_vector_fear, delimiter=",")

In [202]:
texture_test_vector_fear  = get_texture_features(series_fear, pca_cartoon, std_scaler)

(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)


In [205]:
a, b , c = texture_test_vector_fear.shape
texture_test_vector_fear = texture_test_vector_fear.reshape(a,c)

In [207]:
texture_test_vector_fear.shape

(19, 168100)

In [208]:
np.savetxt('im_newT_texturearray_fear.csv', texture_test_vector_fear, delimiter=",")

In [220]:
cartoon_test_vector_sadness  = get_cartoon_features(series_sadness, pca_cartoon, std_scaler)

(410, 410)
['ck+copynewTfile/cartoon_training_sadness/S011/002/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_sadness/S011/002/cartoonimg20.png']
(410, 410)
['ck+copynewTfile/cartoon_training_sadness/S014/002/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_sadness/S014/002/cartoonimg14.png']
(410, 410)
['ck+copynewTfile/cartoon_training_sadness/S026/002/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_sadness/S026/002/cartoonimg14.png']
(410, 410)
['ck+copynewTfile/cartoon_training_sadness/S064/004/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_sadness/S064/004/cartoonimg12.png']
(410, 410)
['ck+copynewTfile/cartoon_training_sadness/S066/004/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_sadness/S066/004/cartoonimg8.png']
(410, 410)
['ck+copynewTfile/cartoon_training_sadness/S080/005/cartoonimg0.png'
 'ck+copynewTfile/cartoon_training_sadness/S080/005/cartoonimg11.png']
(410, 410)
['ck+copynewTfile/cartoon_training_sadness/S081/002/cartoonimg0.png'
 'ck+copynewTfi

In [223]:
cartoon_test_vector_sadness.shape

(21, 168100)

In [222]:
a, b , c = cartoon_test_vector_sadness.shape
cartoon_test_vector_sadness = cartoon_test_vector_sadness.reshape(a,c)

In [224]:
np.savetxt('im_newT_cartoonarray_sadness.csv', cartoon_test_vector_sadness, delimiter=",")

In [226]:
texture_test_vector_sadness  = get_texture_features(series_sadness, pca_cartoon, std_scaler)

(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)
(410, 410)


In [229]:
texture_test_vector_sadness.shape

(21, 168100)

In [228]:
a, b , c = texture_test_vector_sadness.shape
texture_test_vector_sadness = texture_test_vector_sadness.reshape(a,c)

In [230]:
np.savetxt('im_newT_texturearray_sadness.csv', texture_test_vector_sadness, delimiter=",")

In [213]:
def get_texture_features(series, pca, scaler):
    texture_features = []    
    for s in range(0, len(series)):
        texture_f = get_texture_f(series[s], pca, scaler)        
        texture_features.append(texture_f)        
    texture_features = np.asarray(texture_features)
    return texture_features

In [225]:
def  get_texture_f(s, pca, scaler):    
    warpimgpath = 'ck+copynewTfile/transformed_sadness_images/' + s + '/'
    imagepathfirst = warpimgpath +'/warped_image_procustres_'+ str(0) + '.png'
    imagepath = warpimgpath +'/warped_image_procustres_'+ str(len(os.listdir(warpimgpath))-2) + '.png'
    
    img = cv2.imread(imagepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img2 = img[40:450, 170:580]
    print(img2.shape)
   
    texture_e = "ck+copynewTfile/texture_training_sadness/" + s +"/textureimg"  + str(len(os.listdir(warpimgpath))-2) +".png"
    
    
    
    imtext = []

    texture_images = np.asarray([texture_e])
    for i in texture_images:
        img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)    
        imtext.append(img.flatten())
    
    imtextarray = np.asarray(imtext)  
    

    
    #test_texture_data = imtextarray.reshape(1,-1)    
    #scaled_inst = scaler.transform(test_texture_data)
    #b_texture_instance = pca.transform(scaled_inst)
    
    
    
    return imtextarray#b_texture_instance

In [219]:
def get_cartoon_features(series, pca, scaler):
    cartoon_features = []    
    for s in range(0, len(series)):
        cartoon_f = get_cartoon_f(series[s], pca, scaler)        
        cartoon_features.append(cartoon_f)        
    cartoon_features = np.asarray(cartoon_features)
    return cartoon_features

In [218]:
def  get_cartoon_f(s, pca, scaler):    
    warpimgpath = 'ck+copynewTfile/transformed_sadness_images/' + s + '/'
    imagepathfirst = warpimgpath +'warped_image_procustres_'+ str(0) + '.png'
    imagepath = warpimgpath +'warped_image_procustres_'+ str(len(os.listdir(warpimgpath))-2) + '.png'

    img = cv2.imread(imagepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img2 = img[40:450, 170:580]
    print(img2.shape)

    if not os.path.exists("ck+copynewTfile/cartoon_training_sadness/" + s):
        # Create the directory
        os.makedirs("ck+copynewTfile/cartoon_training_sadness/"+ s)
    if not os.path.exists("ck+copynewTfile/texture_training_sadness/" + s):
        # Create the directory
        os.makedirs("ck+copynewTfile/texture_training_sadness/" +s)
        
    #cartoon, texture = cartoonTexture_grey(img2, 0.2, 50, 0.35, 0.35, 1.0, 0)
    #cv2.imwrite("ck+copynewTfile/cartoon_training_sadness/" + s +"/cartoonimg" + str(len(os.listdir(warpimgpath))-2) +".png", cartoon)
    #cv2.imwrite("ck+copynewTfile/texture_training_sadness/"+ s +"/textureimg" + str(len(os.listdir(warpimgpath))-2)+".png", texture)
    
    #img = cv2.imread(imagepathfirst)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #img2 = img[40:450, 170:580]

    #cartoon, texture = cartoonTexture_grey(img2, 0.2, 50, 0.35, 0.35, 1.0, 0)
    #cv2.imwrite("ck+copynewTfile/cartoon_training_sadness/" + s +"/cartoonimg" + str(0) +".png", cartoon)
    #cv2.imwrite("ck+copynewTfile/texture_training_sadness/" +s +"/textureimg" + str(0)+".png", texture)
    
    
    cartoon_f = "ck+copynewTfile/cartoon_training_sadness/" + s +"/cartoonimg" + str(0) +".png"
    cartoon_e = "ck+copynewTfile/cartoon_training_sadness/" + s +"/cartoonimg" + str(len(os.listdir(warpimgpath))-2) +".png"
    texture_e = "ck+copynewTfile/texture_training_sadness/" + s +"/textureimg"  + str(len(os.listdir(warpimgpath))-2) +".png"
    
    imcart = []
    cartoon_images = np.asarray([cartoon_f, cartoon_e])
    print(cartoon_images)
    for i in cartoon_images:
        img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
        img_mean = img/np.mean(img)
        imcart.append(img_mean.flatten())
    
    imcartoon = np.asarray(imcart)
    imcartooncopy = []
    for i in range(0, len(imcartoon) ):
        imca = imcartoon[i] - imcartoon[0]
        imcartooncopy.append(np.asarray(imca))

    imcartoonarray = np.asarray(imcartooncopy)
    
    test_cartoon_data = imcartoonarray[1].reshape(1,-1)
    
    #scaled_inst = scaler.fit_transform(test_cartoon_data)
    #b_cartoon_instance = pca.transform(scaled_inst)
    
    
    
    return test_cartoon_data  #b_cartoon_instance

In [126]:
def get_struct_features(series, pca, scaler, serie_landmarks, serie_shape_data):
    struct_features = []    
    for s in range(0, len(series)):
        struct_f = get_struct_f(series[s], pca, scaler, serie_landmarks[s], serie_shape_data[s])        
        struct_features.append(struct_f)        
    struct_features = np.asarray(struct_features)
    #serie_landmarks = np.asarray(serie_landmarks)
    return struct_features
    

In [138]:
def get_struct_f(s, pca, scaler, one_serie_landmark, serie_data):
    pathi = 'ck+copynewTfile/transformed_sadness_images/' + s + '/'
    firsti = pathi + '/' +np.sort(os.listdir(pathi))[0]    
    first_frame = np.asarray([one_serie_landmark[0]])    
    first_images = np.asarray([firsti])
    struct_test = get_21_points(first_frame, first_images)
    last_point = get_emotion_intensity(old_and_sadfear_PCA, serie_data)
    structre_test_f =np.append(struct_test,last_point )
    
    #test_struc_data = structre_test_f.reshape(1,-1)    
    #scaled_inst = scaler.transform(test_struc_data)
    

    #b_structure_instance = pca.transform(scaled_inst)
    
    return structre_test_f# b_structure_instance

In [128]:
def get_emotion_intensity(my_data_forPCA, serie_shape_data):
    Ai = my_data_forPCA.max()
    data_2_intensity_div = serie_shape_data.flatten()/Ai
    data_2_intensity_sum =  np.sum(data_2_intensity_div)
    data_2_intensity_final = data_2_intensity_sum/136
    return data_2_intensity_final

In [129]:
def get_21_points(first_frame, first_images ):
    first_feature_point = list_get_feature_point(first_frame, 1, first_images)
    second_feature_point = list_get_feature_point(first_frame, 2, first_images)
    third_feature_point = list_get_feature_point(first_frame, 3, first_images)
    fourth_feature_point = list_get_feature_point(first_frame, 4, first_images)
    five_feature_point = list_get_feature_point(first_frame, 5, first_images)
    six_feature_point = list_get_feature_point(first_frame, 6, first_images)
    seven_feature_point = list_get_feature_point(first_frame, 7, first_images)
    eight_feature_point = list_get_feature_point(first_frame, 8, first_images)
    nine_feature_point = list_get_feature_point(first_frame, 9, first_images)
    ten_feature_point = list_get_feature_point(first_frame, 10, first_images)
    eleven_feature_point = list_get_feature_point(first_frame, 11, first_images)
    twelve_feature_point = list_get_feature_point(first_frame, 12, first_images)
    thirteen_feature_point = list_get_feature_point(first_frame, 13, first_images)
    fourteen_feature_point = list_get_feature_point(first_frame, 14, first_images)
    fifteen_feature_point = list_get_feature_point(first_frame, 15, first_images)
    sixteen_feature_point = list_get_feature_point(first_frame, 16, first_images)
    seventeen_feature_point = list_get_feature_point(first_frame, 17, first_images)
    eigtheen_feature_point = list_get_feature_point(first_frame, 18, first_images)
    nineteen_feature = list_get_feature_point(first_frame, 19, first_images)
    twenty_feature = list_get_feature_point(first_frame, 20, first_images)
    twentyone_feature = list_get_feature_point(first_frame, 21, first_images)
    
    structure_vector = first_feature_point
    structure_vector = np.vstack((structure_vector, second_feature_point))
    structure_vector = np.vstack((structure_vector, third_feature_point))
    structure_vector = np.vstack((structure_vector, fourth_feature_point))
    structure_vector = np.vstack((structure_vector, five_feature_point))
    structure_vector = np.vstack((structure_vector, six_feature_point))
    structure_vector = np.vstack((structure_vector, seven_feature_point))
    structure_vector = np.vstack((structure_vector, eight_feature_point))
    structure_vector = np.vstack((structure_vector, nine_feature_point))
    structure_vector = np.vstack((structure_vector, ten_feature_point))
    structure_vector = np.vstack((structure_vector, eleven_feature_point))
    structure_vector = np.vstack((structure_vector, twelve_feature_point))
    structure_vector = np.vstack((structure_vector, thirteen_feature_point))
    structure_vector = np.vstack((structure_vector, fourteen_feature_point))
    structure_vector = np.vstack((structure_vector, fifteen_feature_point))
    structure_vector = np.vstack((structure_vector, sixteen_feature_point))
    structure_vector = np.vstack((structure_vector, seventeen_feature_point))
    structure_vector = np.vstack((structure_vector, eigtheen_feature_point))
    structure_vector = np.vstack((structure_vector, nineteen_feature))
    structure_vector = np.vstack((structure_vector, twenty_feature))
    structure_vector = np.vstack((structure_vector, twentyone_feature))
    
    structre_features = np.asarray(structure_vector).flatten().T
    return structre_features


In [58]:
def get_shape_features(series, pca, scaler):
    shape_features = []
    serie_landmarks = []
    serie_shape_data = []
    for s in series:
        feature_landmarks, shape_f, test_s_data = get_shape_f(s, pca, scaler)        
        shape_features.append(shape_f)
        serie_landmarks.append(feature_landmarks)
        serie_shape_data.append(test_s_data)
    shape_features = np.asarray(shape_features)
    serie_shape_data = np.asarray(serie_shape_data)
    return serie_landmarks, shape_features, serie_shape_data

In [71]:
def get_shape_f(serie, pca,scaler):
    image_path =  '../../ck+/extended-cohn-kanade-images/cohn-kanade-images/' + serie
    land_path = '../../ck+/Landmarks/' + serie
    
    test_images = []
    test_land = []
    
    files = Path(image_path).rglob("*")
    for file in files:       
        if len(os.path.normpath(file).split(os.path.sep)[7].split('_')) == 3:
            test_images.append(str(file))
            
    files = Path(land_path).rglob("*")
    for file in files:
        if len(os.path.normpath(file).split(os.path.sep)[6].split('_')) == 4:
            test_land.append(str(file))
    
    test_images = np.sort(test_images)
    test_land = np.sort(test_land)
    
    transformed_landmarks_total, test_shape_data = get_test_shape_data(test_images, test_land, serie)
    test_shape_data = test_shape_data.reshape(1,-1)
    
    #scaled_inst = scaler.transform(test_shape_data)
    #b_shape_instance = pca.transform(scaled_inst)  
    b_shape_instance =0
    
    return transformed_landmarks_total, b_shape_instance, test_shape_data

In [100]:
def get_test_shape_data(test_images, test_land, serie):
    landmarks_seqXY = get_landmarks_of_folder(test_land)
    cant_de_frames = len(landmarks_seqXY)
    intercalated_data = intercalate_data_of_vectors(landmarks_seqXY, 0)
    pathi = 'ck+copynewTfile/transformed_sadness_images/' + serie + '/'
    
    if not os.path.exists(pathi):
        # Create the directory
        os.makedirs(pathi)
    reference_image_path = 'ck+copynewTfile/reference_image.png'
    my_landmark_for_ref = pd.read_csv('ck+copynewTfile/cknewT_landmark_intercalated_data.csv',header = None)
    reference_land = my_landmark_for_ref.iloc[0]
    transformed_landmarks_total = align_and_transform_vector_of_images(reference_image_path, reference_land, intercalated_data, test_images, pathi)
    transformed_final = transformed_landmarks_total[0].flatten('F')
    for i in range (1, len(transformed_landmarks_total)):
        transformed_final = np.vstack((transformed_final, transformed_landmarks_total[i].flatten('F')))
    nd =0
    my_data_aligned_diff =[]
    for i in range(0, cant_de_frames):#51
        my_data_aligned_diff.append(transformed_final[i]- transformed_final[0]) #no dice nada de abs
    my_data_aligned_diff = np.asarray(my_data_aligned_diff)

    test_shape_data = my_data_aligned_diff[len(my_data_aligned_diff) -2] #-2 instead of -1 for penultimate frame
    return transformed_landmarks_total, test_shape_data